## Assignment 4

We will use pre-populated accelerometer data to learn how feature engineering boosts model performance.
We will use Apache SystemML to implement Discrete Fourier Transformation on the accelerometer sensor time series, therefore transforming the dataset from the time to the frequency domain.
Then, we will train an Apache SparkML classification model to obtain model predictions on the transformed data.

Apache SystemML (now renamed as SystemDS) is a system that accelerates exploratory algorithm development for large-scale machine learning problems. SystemML provides a high-level language to quickly implement and run machine learning algorithms on Spark. SystemML's cost-based optimizer takes care of low-level decisions about how to use Spark’s parallelism, allowing users to focus on the algorithm and the real-world problem that the algorithm is trying to solve. 

See more here:
https://databricks.com/session/inside-apache-systemml
https://researcher.watson.ibm.com/researcher/view_group.php?id=3174

In [2]:
!pip install pyspark==2.4.5

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 217.8 MB 11 kB/s s eta 0:00:01��██▍     | 179.6 MB 33.0 MB/s eta 0:00:02
     |████████████████████████████████| 197 kB 89.0 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257928 sha256=a375236b7a0a6aa92c9852e39d3be4939830f768f9470e8d1b0a2ba40215cb7b
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/01/c0/03/1c241c9c482b647d4d99412a98a5c7f87472728ad41ae55e1e
Successfully built pyspark


In [3]:
!pip install https://github.com/IBM/coursera/blob/master/systemml-1.3.0-SNAPSHOT-python.tar.gz?raw=true

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 9.9 MB 20.4 MB/s eta 0:00:01
  Created wheel for systemml: filename=systemml-1.3.0-py3-none-any.whl size=9882972 sha256=edf4594520c9c890eb66674aa9048d10be9749162195ccf4f7d50d4bbeaa73dc
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/ed/96/15/1042ed0087d53c21a17788d99d5581169482cfe683f1f6e60a
Successfully built systemml


In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType
from pyspark.sql import SparkSession

In [5]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession \
    .builder \
    .getOrCreate()



So the first thing we need to ensure is that we are on the latest version of SystemML, which is 1.3.0 (as of 20th March'19) Please use the code block below to check if you are already on 1.3.0 or higher. 1.3 contains a necessary fix, that's we are running against the SNAPSHOT


In [6]:
!mkdir -p /home/dsxuser/work/systemml


mkdir: cannot create directory ‘/home/dsxuser’: Permission denied


In [7]:
from systemml import MLContext, dml
ml = MLContext(spark)
ml.setConfigProperty("sysml.localtmpdir", "mkdir /home/dsxuser/work/systemml")
print(ml.version())
    
if not ml.version() == '1.3.0-SNAPSHOT':
    raise ValueError('please upgrade to SystemML 1.3.0, or restart your Kernel (Kernel->Restart & Clear Output)')

1.3.0-SNAPSHOT


In [8]:
!wget https://github.com/IBM/coursera/blob/master/coursera_ml/shake.parquet?raw=true
!mv shake.parquet?raw=true shake.parquet

--2021-11-05 21:08:12--  https://github.com/IBM/coursera/blob/master/coursera_ml/shake.parquet?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/claimed/blob/master/coursera_ml/shake.parquet?raw=true [following]
--2021-11-05 21:08:12--  https://github.com/IBM/claimed/blob/master/coursera_ml/shake.parquet?raw=true
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/IBM/claimed/raw/master/coursera_ml/shake.parquet [following]
--2021-11-05 21:08:12--  https://github.com/IBM/claimed/raw/master/coursera_ml/shake.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/claimed/master/coursera_ml/shake.parquet [following]
--2021-11-05 21:08:13--  https://

Now it’s time to read the sensor data and create a temporary query table.

In [9]:
df = spark.read.parquet('shake.parquet')

In [10]:
df.show()

+-----+---------+-----+-----+-----+
|CLASS| SENSORID|    X|    Y|    Z|
+-----+---------+-----+-----+-----+
|    2| qqqqqqqq| 0.12| 0.12| 0.12|
|    2|aUniqueID| 0.03| 0.03| 0.03|
|    2| qqqqqqqq|-3.84|-3.84|-3.84|
|    2| 12345678| -0.1| -0.1| -0.1|
|    2| 12345678|-0.15|-0.15|-0.15|
|    2| 12345678| 0.47| 0.47| 0.47|
|    2| 12345678|-0.06|-0.06|-0.06|
|    2| 12345678|-0.09|-0.09|-0.09|
|    2| 12345678| 0.21| 0.21| 0.21|
|    2| 12345678|-0.08|-0.08|-0.08|
|    2| 12345678| 0.44| 0.44| 0.44|
|    2|    gholi| 0.76| 0.76| 0.76|
|    2|    gholi| 1.62| 1.62| 1.62|
|    2|    gholi| 5.81| 5.81| 5.81|
|    2| bcbcbcbc| 0.58| 0.58| 0.58|
|    2| bcbcbcbc|-8.24|-8.24|-8.24|
|    2| bcbcbcbc|-0.45|-0.45|-0.45|
|    2| bcbcbcbc| 1.03| 1.03| 1.03|
|    2|aUniqueID|-0.05|-0.05|-0.05|
|    2| qqqqqqqq|-0.44|-0.44|-0.44|
+-----+---------+-----+-----+-----+
only showing top 20 rows



In [11]:
!pip install pixiedust

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 197 kB 28.2 MB/s eta 0:00:01
  Created wheel for pixiedust: filename=pixiedust-1.1.19-py3-none-any.whl size=321803 sha256=9a8f2ce0bcbb0a773167d5955168bd84007e324f53566f434cfa6755fe916c6f
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/05/07/e7/8aca0e820027a63157a916424fd748fb2a2a3e71de5e08eeb8
Successfully built pixiedust


In [12]:
import pixiedust

display(df)

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table SPARK_PACKAGES created successfully
Table USER_PREFERENCES created successfully
Table service_connections created successfully


DataFrame[CLASS: bigint, SENSORID: string, X: double, Y: double, Z: double]

In [13]:
df.createOrReplaceTempView("df")

We’ll use Apache SystemML to implement Discrete Fourier Transformation. This way all computation continues to happen on the Apache Spark cluster for advanced scalability and performance.

As you’ve learned from the lecture, implementing Discrete Fourier Transformation in a linear algebra programming language is simple. Apache SystemML DML is such a language and as you can see the implementation is straightforward and doesn’t differ too much from the mathematical definition (Just note that the sum operator has been swapped with a vector dot product using the %*% syntax borrowed from R
):

<img style="float: left;" src="https://wikimedia.org/api/rest_v1/media/math/render/svg/1af0a78dc50bbf118ab6bd4c4dcc3c4ff8502223">



In [14]:
dml_script = '''
PI = 3.141592654
N = nrow(signal)

n = seq(0, N-1, 1)
k = seq(0, N-1, 1)

M = (n %*% t(k))*(2*PI/N)

Xa = cos(M) %*% signal
Xb = sin(M) %*% signal

DFT = cbind(Xa, Xb)
'''

Now it’s time to create a function which takes a single row Apache Spark data frame as argument (the one containing the accelerometer measurement time series for one axis) and returns the Fourier transformation of it. In addition, we are adding an index column for later joining all axis together and renaming the columns to appropriate names. The result of this function is an Apache Spark DataFrame containing the Fourier Transformation of its input in two columns. 


In [15]:
from pyspark.sql.functions import monotonically_increasing_id

def dft_systemml(signal, name):
    prog = dml(dml_script).input('signal', signal).output('DFT')
    
    return (

    #execute the script inside the SystemML engine running on top of Apache Spark
    ml.execute(prog) 
     
         #read result from SystemML execution back as SystemML Matrix
        .get('DFT') 
     
         #convert SystemML Matrix to ApacheSpark DataFrame 
        .toDF() 
     
         #rename default column names
        .selectExpr('C1 as %sa' % (name), 'C2 as %sb' % (name)) 
     
         #add unique ID per row for later joining
        .withColumn("id", monotonically_increasing_id())
    )
        


Now it’s time to create individual DataFrames containing only a subset of the data. We filter simultaneously for accelerometer each sensor axis and one for each class. This means we’ll get 6 DataFrames. We can implement this using the relational API of DataFrames or SparkSQL. 
Here we should use class 1 and 2 and not 0 and 1. Let's make sure that each DataFrame has only ONE colum (only the measurement, eg. not CLASS column)

In [21]:
# spark.sql("select x from df where class = 1").show()

In [22]:
x0 = spark.sql("select x from df where class = 1") # create a DataFrame containing only measurements of class 0 from the x axis
y0 = spark.sql("select y from df where class = 1") # create a DataFrame containing only measurements of class 0 from the y axis
z0 = spark.sql("select z from df where class = 1") # create a DataFrame containing only measurements of class 0 from the z axis
x1 = spark.sql("select x from df where class = 2") # create a DataFrame containing only measurements of class 1 from the x axis
y1 = spark.sql("select y from df where class = 2") # create a DataFrame containing only measurements of class 1 from the y axis
z1 = spark.sql("select z from df where class = 2") # create a DataFrame containing only measurements of class 1 from the z axis

Since we’ve created this cool DFT function before, we can just call it for each of the 6 DataFrames now. And since the result of this function call is a DataFrame again we can use the pyspark best practice in simply calling methods on it sequentially. So what we are doing is the following:

- Calling DFT for each class and accelerometer sensor axis.
- Joining them together on the ID column. 
- Re-adding a column containing the class index.
- Stacking both Dataframes for each classes together



In [23]:
from pyspark.sql.functions import lit

df_class_0 = dft_systemml(x0,'x') \
    .join(dft_systemml(y0,'y'), on=['id'], how='inner') \
    .join(dft_systemml(z0,'z'), on=['id'], how='inner') \
    .withColumn('class', lit(0))
    
df_class_1 = dft_systemml(x1,'x') \
    .join(dft_systemml(y1,'y'), on=['id'], how='inner') \
    .join(dft_systemml(z1,'z'), on=['id'], how='inner') \
    .withColumn('class', lit(1))

df_dft = df_class_0.union(df_class_1)

df_dft.show()

SystemML Statistics:
Total execution time:		0.470 sec.
Number of executed Spark inst:	0.


SystemML Statistics:
Total execution time:		0.221 sec.
Number of executed Spark inst:	0.


SystemML Statistics:
Total execution time:		0.192 sec.
Number of executed Spark inst:	0.


[Stage 21:>                                                         (0 + 1) / 1]
[Stage 23:>                                                         (0 + 1) / 1]
SystemML Statistics:
Total execution time:		14.966 sec.
Number of executed Spark inst:	6.

                                                                                

[Stage 29:>                                                         (0 + 1) / 1]
                                                                                
[Stage 31:>                                                         (0 + 1) / 1]
SystemML Statistics:
Total execution time:		11.022 sec.
Number of executed Spark inst:	6.

                                                          

Let's create a VectorAssembler which consumes the newly created DFT columns and produces a column “features”


In [24]:
from pyspark.ml.feature import VectorAssembler

In [25]:
vectorAssembler = VectorAssembler(inputCols=["xa","xb","ya","yb","za","zb"],
                                  outputCol="features")

Let's instantiate a classifier from the SparkML package and assign it to the classifier variable. Let's make sure to set the “class” column as target.


In [26]:
from pyspark.ml.classification import GBTClassifier

In [27]:
classifier = GBTClassifier(labelCol="class", featuresCol="features", maxIter=10)

Let’s train and evaluate…


In [28]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [29]:
model = pipeline.fit(df_dft)

In [30]:
prediction = model.transform(df_dft)

In [31]:
prediction.show()

+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
| id|                  xa|                  xb|                  ya|                  yb|                  za|                  zb|class|            features|       rawPrediction|         probability|prediction|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
| 26| 0.03912775214058598|-0.09049016952668232| 0.03912775214058598|-0.09049016952668232| 0.03912775214058598|-0.09049016952668232|    0|[0.03912775214058...|[0.88066422361181...|[0.85337596077307...|       0.0|
| 29|0.006260524476137005|-0.05765058448048809|0.006260524476137005|-0.05765058448048809|0.006260524476137005|-0.05765058448048809|    0|[0.006260524476

In [32]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

binEval = MulticlassClassificationEvaluator().setMetricName("accuracy").setPredictionCol("prediction").setLabelCol("class")
binEval.evaluate(prediction) 

0.9981761070017225

In [33]:
!rm -Rf a2_m4.json

In [34]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m4.json')